In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [3]:
df = pd.read_csv('ratings_Electronics.csv')

df.columns=['userId','productId','rating','timestamp']
display(df[0:5])
print(len(df))

,userId,productId,rating,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


7824481


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from surprise import Dataset, Reader, SVD,NormalPredictor, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split as surprise_train_test_split

reader = Reader(rating_scale=(0,5)) # rating scale range
data = Dataset.load_from_df(df[['userId', 'productId', 'rating']], reader)

In [5]:
#Total Users/Products
print(len(df['userId'].unique().tolist()))
print(len(df['productId'].unique().tolist()))

4201696
476001


###Surprise module for Rec system algorithms - start with SVD then explore others to improve performance###

In [10]:
train_df, test_df = surprise_train_test_split(data, test_size=0.20)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824481 entries, 0 to 7824480
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     object 
 1   productId  object 
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 238.8+ MB


[SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:

In [9]:
# df.value_counts().plot(kind='bar')
plt.show()

KeyboardInterrupt: 

In [7]:
# algo = SVD()
algo = NormalPredictor()
# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.7799  1.7786  1.7775  1.7791  1.7794  1.7789  0.0008  
MAE (testset)     1.3629  1.3616  1.3608  1.3628  1.3621  1.3620  0.0008  
Fit time          5.90    7.61    7.47    7.51    7.45    7.19    0.64    
Test time         8.38    8.01    6.16    8.06    4.44    7.01    1.50    


{'test_rmse': array([1.77987139, 1.77860443, 1.77747047, 1.77911963, 1.77942712]),
 'test_mae': array([1.36289624, 1.36158103, 1.36077735, 1.36281816, 1.36206326]),
 'fit_time': (5.903126955032349,
  7.605234861373901,
  7.468047380447388,
  7.509538650512695,
  7.445844650268555),
 'test_time': (8.377801895141602,
  8.0125412940979,
  6.15601110458374,
  8.059755802154541,
  4.437545537948608)}

In [8]:
algo.fit(train_df)
predictions = algo.test(test_df)

predictions.head()

NameError: name 'train_df' is not defined